In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pickle

from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest

from sklearn.model_selection import RepeatedKFold, RepeatedStratifiedKFold
from sklearn.model_selection import RandomizedSearchCV

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

from models import build_model, compare_result

sessions = pd.read_csv("merged_dataset", sep=' ')

## Usunięcie redundantnych atrybutów

In [ ]:
sessions = sessions.drop(['session_id', 'user_id', 'unique_categories'], axis=1)

## One-hot encoding dla danych zawierających labele, a nie wartości

In [ ]:
sessions = pd.get_dummies(sessions, columns = ['offered_discount', 'weekday'])

## Podział zbioru na podzbiory: testowy i treningowy

12% danych testowych i 88% danych treningowych

In [ ]:
seed = 116
test_size = 0.12

X = sessions.drop('purchase', axis=1)
Y = sessions[['purchase']]
records_num = len(sessions)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=test_size, random_state=seed)
print('Size of entire dataset: ', records_num)
print('Size of training set: ', len(X_train))
print('Size of test set: ', len(X_test))

In [ ]:
X_train.head()

## Usunięcie powiązanych ze sobą atrybutów

In [ ]:
plt.rcParams['figure.figsize'] = [50, 50]
plt.rcParams.update({'font.size': 32})

train = pd.concat([X_train, Y_train], axis=1)

sns.heatmap(
    train.corr(method = 'spearman'),
    xticklabels = train.columns,
    yticklabels = train.columns,
    annot=True,
    fmt='0.1g'
)

//Wnioski o korelacji rangowej spearmana i jeżeli wyjdą skorelowane atrybuty to musimy zrobić chi kwadrat by zobaczyć, który trzeba usunąć

## Wykonanie testu chi kwadrat na wszystkich atrybutach

In [ ]:
list_one =[]
feature_ranking = SelectKBest(chi2, k=5)
fit = feature_ranking.fit(X_train, Y_train)

for i, (score, feature) in enumerate(zip(feature_ranking.scores_, X_train.columns)):
    list_one.append((score, feature))
    
dfObj = pd.DataFrame(list_one) 
dfObj = dfObj.sort_values(by=[0], ascending = False)

display(dfObj)

//usuwamy skorelowane atrybuty i te bliskie zeru

## Prezentacja ostatecznych wejść i wyjść konstruowanych modelów

VVVV - trzeba przeredagować

Zatem naszymi danymi wejściowymi modelu będą następujące atrybuty:

duration - długość trwania sesji w sekundach
click_rate - liczba zdarzeń(event'ów) do aktualnego rekordu na minutę
weekday_0.0 - weekday_6.0 - one hot encoding dni tygodnia
hour - godzina
offered_discount_10, 15, 20 - one hot encoding 10, 15 oraz 20%-owej zniżki
weekend - wartość boolowska odpowiadająca na pytanie czy sesja trwa w weekend
Ponadto dodaliśmy nowe atrybuty:

last_session_purchase - odpowiada na pytanie czy ostatnia sesja zakończyła się zakupem
price - cena przeglądanego produktu
Natomiast zmienną celu jest prawdopoboieństwo tego, że sesja zakończy się zakupem. Model będzie zwracał, że sesja zakończy się zakupem jeśli prawdopodobieństwo zakupu będzie większe od pewnej wartości granicznej.

In [ ]:
sessions = pd.read_csv("merged_dataset", sep=' ')

result_dict = {}

dropped_cols_with_corelated_attr = list(dfObj[dfObj[0] < 1][1])
dropped_cols_with_corelated_attr.extend(['session_id', 'user_id', 'unique_categories'])

dropped_cols_without_corelated_attr = dropped_cols_with_corelated_attr[:]
dropped_cols_without_corelated_attr.extend(['unique_item_views', 'item_views'])

onehot_cols = ['offered_discount', 'weekday']

## Model bazowy - regresja logistyczna

In [ ]:
def logistic_fn_tuning(X_train, Y_train):
    # hyperparameters tuning
    model = LogisticRegression(max_iter=500)
    # define evaluation
    cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=seed)

    # define search space
    space = dict()
    space['solver'] = ['newton-cg', 'lbfgs', 'liblinear']
    space['penalty'] = ['l2']
    space['C'] = [100, 10, 1.0, 0.1, 0.01]

    cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=seed)
    search = RandomizedSearchCV(model, space, scoring='accuracy', n_jobs=-1, cv=cv, random_state=seed)
    search.fit(X_train, Y_train)
    
    print(search.best_estimator_) # model = LogisticRegression(C=0.1, max_iter=500, solver='newton-cg')
    return search.best_estimator_

def logistic_fn(X_train, Y_train):
    model = LogisticRegression(C=0.1, max_iter=500, solver='newton-cg')
    model.fit(X_train, Y_train)
    
    return model

result_dict['Logistic Regression with corelated attributes'] = \
    build_model(logistic_fn, sessions, onehot_cols, dropped_cols_with_corelated_attr, 'purchase', 
                seed, test_size)

result_dict['Logistic Regression without corelated attributes'] = \
    build_model(logistic_fn, sessions, onehot_cols, dropped_cols_without_corelated_attr, 'purchase', seed, test_size, output = './microservice/models/logistic_reg.pkl')

## Zaawansowany model docelowy - regresja logistyczna

In [ ]:
def random_forest_fn_tuning(X_train,Y_train):
    model = RandomForestClassifier()
    
    # define evaluation
    cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=seed)
    
    # define search space
    space = dict()
    space['n_estimators'] = [100, 200, 250, 300, 350, 400, 500, 800, 1200]
    space['max_depth'] = [5, 8, 10, 15, 20, 25, 30]
    space['min_samples_split'] = [2, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 100]
    space['min_samples_leaf'] = [1, 2, 5, 10] 
    
    cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=seed)
    search = RandomizedSearchCV(model, space, n_iter=50, scoring='accuracy', n_jobs=-1, cv=cv, random_state=seed)
    search.fit(X_train, Y_train)
    
    print(search.best_estimator_) # RandomForestClassifier(max_depth=25, min_samples_split=15, n_estimators=300)
    
    return search.best_estimator_

def random_forest_fn(X_train,Y_train):
    model = RandomForestClassifier(max_depth=9, min_samples_split=15, n_estimators=300) 
    model.fit(X_train, Y_train)
    
    return model
    
result_dict['Random_Forest with corelated attributes'] = \
    build_model(random_forest_fn, sessions, onehot_cols, dropped_cols_with_corelated_attr,
                'purchase', seed)
result_dict['Random_Forest without corelated attributes'] = \
    build_model(random_forest_fn, sessions, onehot_cols, dropped_cols_without_corelated_attr, 'purchase', seed, output='./microservice/models/random_forest.pkl')

## Porównanie wyników prezentowanych przez modele

In [ ]:
scope = ['training', 'test']
records = []
for name in result_dict:
    for option in scope:
        record = []
        for key in result_dict[name][option]:
            record.append(result_dict[name][option][key])
        records.append(record)
            
df = pd.DataFrame.from_records(records)
df.columns = ['Accuracy', 'Precision', 'Recall', 'F1_score']
df.index = ['LR with corelated attributes / training data', 'LR with corelated attributes / test data', 
            'LR without corelated attributes / training data', 'LR without corelated attributes / test data', 
            'RF with corelated attributes / training data', 'RF with corelated attributes / test data', 
            'RF without corelated attributes / training data', 'RF without corelated attributes / test data']
df

## Analiza uzyskanych wyników

VVVV - przeredagować 


Zaproponowaliśmy 2 modele tj. model bazowy będący regresją logistyczną oraz bardziej zaawansowany model docelowy będący modelem lasów losowych. Dodatkowo dla każdego z modeli rozpatrzyliśmy 2 przypadki:

uwzględnienie skorelowanych atrybutów
usunięcie skorelowanych atrybutów tj. usunięcie atrybutów 'unique_item_views' oraz 'item_views' i zachowanie atrybutu 'duration'
Dla każdego z powyższych modeli obliczyliśmy miarę Accuracy oraz F1-score. Pierwsza wartość opisuje liczbę trafnych przewidywań do wszystkich próbek. Natomiast F1-score jest średnią harmoniczną z recall oraz precision.

Zdecydowaliśmy się jednak, że naszą analityczną miarą sukcesu będzie F1-score, ponieważ nasze zadanie biznesowe polega na "klasyfikacji sesji, które zakończą się zakupem co umożliwi konsultantom szybsze rozwiązywanie problemów". Z tego można wywnioskować, że zależy nam na tym aby:

liczba przypadków nieprawidłowo sklasyfikowanych jako sesje niekupujące była jak najmniejsza, dzięki czemu nie pomijamy sesji kupujących, które będą mogły wymagać interwencji konsultantów
liczba przypadków sklasyfikowanych niepoprawnie jako sesja kupująca była jak najmniejsza dzięki czemu oszczędzamy czas konsultantów
Z tego wynika, że bardziej zależy nam nam na False Negative'ach oraz False Positive'ach niż na True Positive'ach oraz True Negative'ach

Analizując wyniki, możemy zauważyć że model Random Forest ma większą miarę F1 od regresji logistycznej zarówno w przypadku zachowania skorelowanych atrybutów jak i w przypadku usnięcia ich.
W obu przypadkach postanowiliśmy usunąć atrybuty skorelowane tj. 'unique_item_views' oraz 'item_views' z racji na wartości F1.